In [15]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split


# Definición de rutas
BASE = Path.cwd().parent 
print(f"Base directory: {BASE}")
RAW_DATA = BASE / "data" / "raw"  
METADATA = RAW_DATA / "plasticc_test_metadata.csv"

Base directory: c:\Users\David\___utad\4_Cuarto\TFG\proyecto_PLAsTiCC


In [16]:
# Lista todos los archivos que contienen 'batch' en RAW_DATA
batch_files = []
for file in RAW_DATA.iterdir():
    if 'batch' in file.name.lower():
        batch_files.append(file)
        print(file.name)

print(f"\nTotal de archivos con 'batch': {len(batch_files)}")

plasticc_test_set_batch1.csv
plasticc_test_set_batch10.csv
plasticc_test_set_batch11.csv
plasticc_test_set_batch2.csv
plasticc_test_set_batch3.csv
plasticc_test_set_batch4.csv
plasticc_test_set_batch5.csv
plasticc_test_set_batch6.csv
plasticc_test_set_batch7.csv
plasticc_test_set_batch8.csv
plasticc_test_set_batch9.csv

Total de archivos con 'batch': 11


In [17]:
# Cargar el primer archivo batch y obtener los object_id únicos
first_batch = pd.read_csv(batch_files[1])
unique_object_ids = first_batch['object_id'].unique()

print(f"Archivo cargado: {batch_files[0].name}")
print(f"Total de object_id únicos: {len(unique_object_ids)}")

Archivo cargado: plasticc_test_set_batch1.csv
Total de object_id únicos: 345996


In [18]:
# Filtrar metadatos para los object_id únicos del primer batch
metadata_df = pd.read_csv(METADATA)

metadata_filtered = metadata_df[metadata_df['object_id'].isin(unique_object_ids)]
target_counts_filtered = metadata_filtered['true_target'].value_counts().sort_index()

print("Conteo de ocurrencias por true_target para los object_id del primer batch:")
print(target_counts_filtered)
print(f"\nTotal de objetos filtrados: {len(metadata_filtered)}")

Conteo de ocurrencias por true_target para los object_id del primer batch:
true_target
6         125
15       1326
16       9774
42      98951
52       6333
53        152
62      17427
64         17
65       9304
67       3989
88       9945
90     164242
92      19578
95       3572
991        45
992       156
993       945
994       115
Name: count, dtype: int64

Total de objetos filtrados: 345996


In [13]:
# Procesar todos los archivos batch y contar true_target por archivo
batch_results = {}

for batch_file in batch_files:
    # Cargar el archivo batch
    batch_df = pd.read_csv(batch_file)
    batch_object_ids = batch_df['object_id'].unique()
    
    # Filtrar metadatos para los object_id de este batch
    batch_metadata_filtered = metadata_df[metadata_df['object_id'].isin(batch_object_ids)]
    batch_target_counts = batch_metadata_filtered['true_target'].value_counts().sort_index()
    
    # Guardar resultados
    batch_results[batch_file.name] = {
        'total_objects': len(batch_object_ids),
        'target_counts': batch_target_counts
    }
    
    print(f"\n{batch_file.name}:")
    print(f"Total de object_id únicos: {len(batch_object_ids)}")
    print("Conteo por true_target:")
    print(batch_target_counts)



print(f"\n\nResumen procesado: {len(batch_results)} archivos batch")


plasticc_test_set_batch1.csv:
Total de object_id únicos: 32926
Conteo por true_target:
true_target
6          6
15        68
16         3
42     15986
52       807
53         6
62      2194
64         2
65        61
67       362
88       363
90     12640
92       142
95        98
991        9
992       31
993      142
994        6
Name: count, dtype: int64

plasticc_test_set_batch10.csv:
Total de object_id únicos: 345996
Conteo por true_target:
true_target
6         125
15       1326
16       9774
42      98951
52       6333
53        152
62      17427
64         17
65       9304
67       3989
88       9945
90     164242
92      19578
95       3572
991        45
992       156
993       945
994       115
Name: count, dtype: int64

plasticc_test_set_batch11.csv:
Total de object_id únicos: 345996
Conteo por true_target:
true_target
6         127
15       1336
16       9637
42      98738
52       6170
53        153
62      17249
64         14
65       9352
67       4050
88      10011
90  

In [14]:
import pandas as pd

# Crear un DataFrame a partir de batch_results para guardar en CSV

# Crear una lista para almacenar los datos
data_for_csv = []

for batch_name, batch_info in batch_results.items():
    # Para cada batch, crear una fila con el nombre del batch, total de objetos
    # y el conteo de cada true_target
    row = {'batch_file': batch_name, 'total_objects': batch_info['total_objects']}
    
    # Agregar el conteo de cada true_target como columnas separadas
    for target, count in batch_info['target_counts'].items():
        row[f'target_{target}'] = count
    
    data_for_csv.append(row)

# Crear DataFrame y guardarlo como CSV
results_df = pd.DataFrame(data_for_csv)
results_df = results_df.fillna(0).astype(int, errors='ignore')  # Rellenar NaN con 0 y convertir a int donde sea posible

# Definir la ruta de salida
output_file = BASE / "data" / "processed" / "batch_results_summary.csv"
output_file.parent.mkdir(parents=True, exist_ok=True)

# Guardar el archivo
results_df.to_csv(output_file, index=False)
print(f"Resultados guardados en: {output_file}")
print(f"Dimensiones del archivo: {results_df.shape}")

Resultados guardados en: c:\Users\David\___utad\4_Cuarto\TFG\proyecto_PLAsTiCC\data\processed\batch_results_summary.csv
Dimensiones del archivo: (11, 20)
